## Librerias

In [2]:
import requests as req
import pandas as pd
import time
import selenium
import webdriver_manager
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager


In [3]:
from selenium.webdriver.chrome.options import Options
# opciones del driver

opciones=Options()

# quita la bandera de ser robot
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

opciones.headless=False   # si True, no aparece la ventana (headless=no visible)

opciones.add_argument('--start-maximized')         # comienza maximizado

#opciones.add_extension('driver/adblock.crx')       # adblocker

opciones.add_argument('user-data-dir=selenium')    # mantiene las coockies


#opciones.add_argument('--incognito')              # incognito
 
#opciones.add_argument('--no-proxy-server')                 # sin proxy
#opciones.add_argument('proxy-server=106.122.8.54:3128')    # proxy custom

### CSV

In [4]:
nypd = pd.read_csv('../data/NYPD_Complaint_Data_Current__Year_To_Date_.csv')

In [5]:
nypd.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,261548672,43.0,BRONX,01/07/2023,21:30:00,NaN,(null),COMPLETED,(null),NaN,...,NaN,UNKNOWN,UNKNOWN,D,1026226.0,236504.0,40.815726,-73.848350,"(40.815726, -73.84835)",POINT (-73.84835 40.815726)
1,265065361,69.0,BROOKLYN,06/12/2022,01:55:00,NaN,(null),COMPLETED,(null),NaN,...,NaN,UNKNOWN,UNKNOWN,E,1012661.0,177364.0,40.653454,-73.897609,"(40.65345427, -73.89760855)",POINT (-73.89760855 40.65345427)
2,262612530,110.0,QUEENS,09/09/2022,00:55:00,09/09/2022,01:19:00,COMPLETED,(null),NaN,...,NaN,25-44,WHITE HISPANIC,F,1019164.0,210169.0,40.743481,-73.874004,"(40.7434812638841, -73.8740035373971)",POINT (-73.8740035373971 40.7434812638841)
3,261233867,123.0,STATEN ISLAND,01/02/2023,02:00:00,01/02/2023,02:10:00,COMPLETED,(null),NaN,...,NaN,45-64,WHITE,F,917367.0,125198.0,40.510063,-74.240524,"(40.510063, -74.240524)",POINT (-74.240524 40.510063)
4,264778583,30.0,MANHATTAN,06/29/2021,21:00:00,06/29/2021,21:30:00,COMPLETED,(null),NaN,...,NaN,25-44,WHITE,F,998164.0,241176.0,40.828637,-73.949722,"(40.828637, -73.949722)",POINT (-73.949722 40.828637)


### Me saco la explicación de cada columna 
soy una vaga y no quiero copiar y pegar

In [6]:
column_names = ['CMPLNT_NUM', 'ADDR_PCT_CD', 'BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT', 
                'CMPLNT_TO_TM', 'CRM_ATPT_CPTD_CD', 'HADEVELOPT', 'HOUSING_PSA', 'JURISDICTION_CODE', 
                'JURIS_DESC', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PATROL_BORO', 
                'PD_CD', 'PD_DESC', 'PREM_TYP_DESC', 'RPT_DT', 'STATION_NAME', 'SUSP_AGE_GROUP', 'SUSP_RACE', 
                'SUSP_SEX', 'TRANSIT_DISTRICT', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'X_COORD_CD', 
                'Y_COORD_CD', 'Latitude', 'Longitude', 'Lat_Lon']

column_desc = ['Randomly generated persistent ID for each complaint',
 'The precinct in which the incident occurred',
 'The name of the borough in which the incident occurred',
 'Exact date of occurrence for the reported event (or starting date of occurrence, if CMPLNT_TO_DT exists)',
 'Exact time of occurrence for the reported event (or starting time of occurrence, if CMPLNT_TO_TM exists)',
 'Ending date of occurrence for the reported event, if exact time of occurrence is unknown',
 'Ending time of occurrence for the reported event, if exact time of occurrence is unknown',
 'Indicator of whether crime was successfully completed or attempted, but failed or was interrupted prematurely',
 'Name of NYCHA housing development of occurrence, if applicable',
 'Development Level Code',
 'Jurisdiction responsible for incident. Either internal, like Police(0), Transit(1), and Housing(2); or external(3), like Correction, Port Authority, etc.',
 'Description of the jurisdiction code',
 'Three digit offense classification code',
 'Level of offense: felony, misdemeanor, violation',
 'Specific location of occurrence in or around the premises; inside, opposite of, front of, rear of',
 'Description of offense corresponding with key code',
 'Name of NYC park, playground or greenspace of occurrence, if applicable (state parks are not included)',
 'The name of the patrol borough in which the incident occurred',
 'Three digit internal classification code (more granular than Key Code)',
 'Description of internal classification corresponding with PD code (more granular than Offense Description)',
 'Specific description of premises; grocery store, residence, street, etc.',
 'Date event was reported to police',
 'Transit station name',
 'Suspect’s Age Group',
 'Suspect’s Race Description',
 'Suspect’s Sex Description',
 'Transit district in which the offense occurred.',
 'Victim’s Age Group',
 'Victim’s Race Description',
 'Victim’s Sex Description',
 'X-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104)',
 'Y-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104)',
 'Midblock Latitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326)',
 'Midblock Longitude coordinate for Global Coordinate System, WGS 1984, decimal degrees (EPSG 4326)',
 'Location']




In [7]:


# Crear el dataframe
descripcion = pd.DataFrame()

descripcion['Nombre_Columna']=column_names
descripcion['Descripcion']=column_desc


descripcion

,Nombre_Columna,Descripcion
0,CMPLNT_NUM,Randomly generated persistent ID for each comp...
1,ADDR_PCT_CD,The precinct in which the incident occurred
2,BORO_NM,The name of the borough in which the incident ...
3,CMPLNT_FR_DT,Exact date of occurrence for the reported even...
4,CMPLNT_FR_TM,Exact time of occurrence for the reported even...
5,CMPLNT_TO_DT,Ending date of occurrence for the reported eve...
6,CMPLNT_TO_TM,Ending time of occurrence for the reported eve...
7,CRM_ATPT_CPTD_CD,Indicator of whether crime was successfully co...
8,HADEVELOPT,Name of NYCHA housing development of occurrenc...
9,HOUSING_PSA,Development Level Code


In [8]:
nans, cols, trim_cols = list(nypd.isnull().sum()), list(nypd.columns), list() 

In [9]:
    print(f"List of NAN counts in columns-\n {nans}\n")
    print(f"List of all attributes-\n {cols}\n")
    
    display_df = pd.DataFrame(np.column_stack([cols, nans]), columns=["colnames", "nan_counts"]) #print nans and cols together in a df
    display(display_df)

List of NAN counts in columns-
 [0, 14, 0, 0, 0, 9056, 0, 0, 0, 121418, 0, 0, 0, 0, 0, 0, 0, 0, 116, 0, 0, 0, 0, 0, 0, 0, 127040, 0, 0, 0, 1, 1, 1, 1, 1, 1]

List of all attributes-
 ['CMPLNT_NUM', 'ADDR_PCT_CD', 'BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'CRM_ATPT_CPTD_CD', 'HADEVELOPT', 'HOUSING_PSA', 'JURISDICTION_CODE', 'JURIS_DESC', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PATROL_BORO', 'PD_CD', 'PD_DESC', 'PREM_TYP_DESC', 'RPT_DT', 'STATION_NAME', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude', 'Lat_Lon', 'New Georeferenced Column']



,colnames,nan_counts
0,CMPLNT_NUM,0
1,ADDR_PCT_CD,14
2,BORO_NM,0
3,CMPLNT_FR_DT,0
4,CMPLNT_FR_TM,0
5,CMPLNT_TO_DT,9056
6,CMPLNT_TO_TM,0
7,CRM_ATPT_CPTD_CD,0
8,HADEVELOPT,0
9,HOUSING_PSA,121418


In [10]:
length_df, width_df = nypd.shape

trim_cols=[]

for i, nan_count in enumerate(nans): #for each nan_count in nans
    if (nan_count / length_df) > 0.5: #if ratio of length of df and number of nans in a column is > 0.5
        trim_cols.append(cols[i]) 
    
print(f"Columns identified for trimming- {trim_cols}\n")
    
nypd = nypd.drop(columns=trim_cols) #drop all columns present in the trim_cols list






Columns identified for trimming- ['HOUSING_PSA', 'TRANSIT_DISTRICT']



### Analisis de columnas sobrantes

CMPLNT_TO_DT Y CMPLNT_TO_TM que son 'Ending date of occurrence for the reported event, if exact time of occurrence is unknown',
 'Ending time of occurrence for the reported event, if exact time of occurrence is unknown', respectivamente las voy a eliminar ya que no me interesa fecha de inicio y fin.

 RPT_DT es redundante ya que también muestra el dia que el crimen fue reportado a la policia

'HADEVELOPT' me la cargo

'JURISDICTION_CODE',

 'JURIS_DESC'

'OFNS_DESC'


'PARKS_NM'


PATROL_BORO


'STATION_NAME'

'X_COORD_CD',


 'Y_COORD_CD'

 ADDR_PCT_CD

 
 

In [11]:
nypd = nypd.drop(columns=['CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'RPT_DT', 'LOC_OF_OCCUR_DESC','HADEVELOPT','JURISDICTION_CODE', 'JURIS_DESC','OFNS_DESC', 'PARKS_NM','PATROL_BORO','STATION_NAME','X_COORD_CD', 'Y_COORD_CD','ADDR_PCT_CD'])

In [12]:
nypd.columns

Index(['CMPLNT_NUM', 'ADDR_PCT_CD', 'BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM',
       'CRM_ATPT_CPTD_CD', 'KY_CD', 'LAW_CAT_CD', 'PD_CD', 'PD_DESC',
       'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX',
       'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude',
       'Lat_Lon', 'New Georeferenced Column'],
      dtype='object')

In [13]:
nypd.PREM_TYP_DESC

0                          OTHER
1                         STREET
2                         STREET
3                         STREET
4         RESIDENCE - APT. HOUSE
                   ...          
130194                    STREET
130195      TRANSIT - NYC SUBWAY
130196            GROCERY/BODEGA
130197                    (null)
130198           RESIDENCE-HOUSE
Name: PREM_TYP_DESC, Length: 130199, dtype: object

Renombrar columnas

dfHplusY_retrim.rename(columns={
        'cmplnt_num':'ComplaintID', 
        'cmplnt_fr_dt':'Date',
        'cmplnt_fr_tm':'Time',
        'ky_cd':'Offence Code',
        'ofns_desc':'Description',
        'pd_cd':'Internal Code',
        'pd_desc':'Internal Description',
        'crm_atpt_cptd_cd':'Status',
        'law_cat_cd':'Offence Level',
        'boro_nm':'Borough',
        'addr_pct_cd':'Neighborhood',
        'prem_typ_desc':'Premise Description'}, inplace=True) #replace columns
    dfHplusY_retrim.reset_index(inplace=True) #reset index 

In [14]:
nypd.rename(columns={
        'CMPLNT_NUM':'ComplaintID', 
        'ADDR_PCT_CD':'Neighborhood',
        'BORO_NM':'Borough',
        'CMPLNT_FR_DT':'Date',
        'CMPLNT_FR_TM':'Time',
        'KY_CD':'Offence Code',
        'OFNS_DESC':'Description',
        'PD_CD':'Internal Code',
        'PD_DESC':'Internal Description',
        'CRM_ATPT_CPTD_CD':'Crime_Attempt',
        'LAW_CAT_CD':'Offence_Level',
        'PREM_TYP_DESC':'Premise_Description',
        'SUSP_AGE_GROUP':'Suspect_Age',
        'SUSP_RACE':'Suspect_Race',
        'SUSP_SEX':'Suspect_Sex',
        'VIC_AGE_GROUP':'Victim_Age',
        'VIC_RACE':'Victim_Race',
        'VIC_SEX':'Victim_Sex',

        }, inplace=True) #replace columns


In [15]:
nypd.columns

Index(['ComplaintID', 'Neighborhood', 'Borough', 'Date', 'Time',
       'Crime_Attempt', 'Offence Code', 'Offence_Level', 'Internal Code',
       'Internal Description', 'Premise_Description', 'Suspect_Age',
       'Suspect_Race', 'Suspect_Sex', 'Victim_Age', 'Victim_Race',
       'Victim_Sex', 'Latitude', 'Longitude', 'Lat_Lon',
       'New Georeferenced Column'],
      dtype='object')

In [16]:
nypd.shape

(130199, 21)

Saco el año y el mes

In [17]:
nypd['Year'] = nypd.Date.str[6:] #get first four values of Date.string
nypd['Month'] = nypd.Date.str[:2] #get the next 3 values of Date.string 

In [18]:
nypd.head()

,ComplaintID,Neighborhood,Borough,Date,Time,Crime_Attempt,Offence Code,Offence_Level,Internal Code,Internal Description,...,Suspect_Sex,Victim_Age,Victim_Race,Victim_Sex,Latitude,Longitude,Lat_Lon,New Georeferenced Column,Year,Month
0,261548672,43.0,BRONX,01/07/2023,21:30:00,COMPLETED,107,FELONY,223.0,"BURGLARY,RESIDENCE,NIGHT",...,M,UNKNOWN,UNKNOWN,D,40.815726,-73.848350,"(40.815726, -73.84835)",POINT (-73.84835 40.815726),2023,01
1,265065361,69.0,BROOKLYN,06/12/2022,01:55:00,COMPLETED,113,FELONY,723.0,"FORGERY,DRIVERS LICENSE",...,M,UNKNOWN,UNKNOWN,E,40.653454,-73.897609,"(40.65345427, -73.89760855)",POINT (-73.89760855 40.65345427),2022,06
2,262612530,110.0,QUEENS,09/09/2022,00:55:00,COMPLETED,578,VIOLATION,638.0,"HARASSMENT,SUBD 3,4,5",...,M,25-44,WHITE HISPANIC,F,40.743481,-73.874004,"(40.7434812638841, -73.8740035373971)",POINT (-73.8740035373971 40.7434812638841),2022,09
3,261233867,123.0,STATEN ISLAND,01/02/2023,02:00:00,COMPLETED,341,MISDEMEANOR,321.0,"LARCENY,PETIT FROM AUTO",...,(null),45-64,WHITE,F,40.510063,-74.240524,"(40.510063, -74.240524)",POINT (-74.240524 40.510063),2023,01
4,264778583,30.0,MANHATTAN,06/29/2021,21:00:00,COMPLETED,344,MISDEMEANOR,101.0,ASSAULT 3,...,M,25-44,WHITE,F,40.828637,-73.949722,"(40.828637, -73.949722)",POINT (-73.949722 40.828637),2021,06


Transformar la columna Month

In [19]:
mes_dic = {
    "01": "January",
    "02": "February",
    "03": "March",
    "04": "April",
    "05": "May",
    "06": "June",
    "07": "July",
    "08": "August",
    "09": "September",
    "10": "October",
    "11": "November",
    "12": "December"
}

nypd['Month'] = nypd['Month'].map(mes_dic)


In [38]:
nypd.head()

,index,ComplaintID,Neighborhood,Borough,Date,Time,Crime_Attempt,Offence Code,Offence_Level,Internal Code,...,Suspect_Sex,Victim_Age,Victim_Race,Victim_Sex,Latitude,Longitude,Lat_Lon,New Georeferenced Column,Year,Month
0,0,261548672,43.0,BRONX,01/07/2023,21:30:00,COMPLETED,107,FELONY,223.0,...,M,UNKNOWN,UNKNOWN,D,40.815726,-73.848350,"(40.815726, -73.84835)",POINT (-73.84835 40.815726),2023,January
1,1,265065361,69.0,BROOKLYN,06/12/2022,01:55:00,COMPLETED,113,FELONY,723.0,...,M,UNKNOWN,UNKNOWN,E,40.653454,-73.897609,"(40.65345427, -73.89760855)",POINT (-73.89760855 40.65345427),2022,June
2,2,262612530,110.0,QUEENS,09/09/2022,00:55:00,COMPLETED,578,VIOLATION,638.0,...,M,25-44,WHITE HISPANIC,F,40.743481,-73.874004,"(40.7434812638841, -73.8740035373971)",POINT (-73.8740035373971 40.7434812638841),2022,September
3,3,261233867,123.0,STATEN ISLAND,01/02/2023,02:00:00,COMPLETED,341,MISDEMEANOR,321.0,...,(null),45-64,WHITE,F,40.510063,-74.240524,"(40.510063, -74.240524)",POINT (-74.240524 40.510063),2023,January
4,4,264778583,30.0,MANHATTAN,06/29/2021,21:00:00,COMPLETED,344,MISDEMEANOR,101.0,...,M,25-44,WHITE,F,40.828637,-73.949722,"(40.828637, -73.949722)",POINT (-73.949722 40.828637),2021,June


In [40]:
nypd['Borough']

0                 BRONX
1              BROOKLYN
2                QUEENS
3         STATEN ISLAND
4             MANHATTAN
              ...      
130194         BROOKLYN
130195        MANHATTAN
130196        MANHATTAN
130197            BRONX
130198           QUEENS
Name: Borough, Length: 130199, dtype: object

In [39]:

nypd['Borough_2']=map(lambda x: x.title(), list(nypd['Borough']))

TypeError: object of type 'map' has no len()

In [29]:
nypd.Year.sort_values()

285       1012
2783      1013
5189      1023
7541      1023
622       1023
          ... 
48122     2023
48121     2023
48120     2023
47968     2023
130198    2023
Name: Year, Length: 130199, dtype: object